In [42]:
#from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
#credit_approval = fetch_ucirepo(id=27) 
  
# data (as pandas dataframes) 
#X = credit_approval.data.features 
#y = credit_approval.data.targets 
  
# metadata 
#print(credit_approval.metadata) 
  
# variable information 
#print(credit_approval.variables) 


In [43]:
# Import pandas
import pandas as pd 

# Load dataset
cc_apps = pd.read_csv('https://archive.ics.uci.edu/static/public/27/data.csv', header = None)

# Inspect data
cc_apps.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,A15,A16
1,b,30.83,0,u,g,w,v,1.25,t,t,01,f,g,00202,0,+
2,a,58.67,4.46,u,g,q,h,3.04,t,t,06,f,g,00043,560,+
3,a,24.50,0.5,u,g,q,h,1.5,t,f,0,f,g,00280,824,+
4,b,27.83,1.54,u,g,w,v,3.75,t,t,05,t,g,00100,3,+


In [44]:
# Print summary statistics
cc_apps_description = cc_apps.describe()
print(cc_apps_description)

print('\n')

# Print DataFrame information
cc_apps_info = cc_apps.info()
print(cc_apps_info)

print('\n')

# Inspect missing values in the dataset
print(cc_apps.tail(17))

         0      1    2    3    4    5    6    7    8    9    10   11   12  \
count   679    679  691  685  685  682  682  691  691  691  691  691  691   
unique    3    350  216    4    4   15   10  133    3    3   24    3    4   
top       b  22.67  1.5    u    g    c    v    0    t    f    0    f    g   
freq    468      9   21  519  519  137  399   70  361  395  395  374  625   

           13   14   15  
count     678  691  691  
unique    171  241    3  
top     00000    0    -  
freq      132  295  383  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 691 entries, 0 to 690
Data columns (total 16 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   0       679 non-null    object
 1   1       679 non-null    object
 2   2       691 non-null    object
 3   3       685 non-null    object
 4   4       685 non-null    object
 5   5       682 non-null    object
 6   6       682 non-null    object
 7   7       691 non-null    object
 8   8       691 

In [45]:
import sklearn.model_selection

# Import train_test_split
from sklearn.model_selection import train_test_split

# Drop the features 11 and 13
cc_apps = cc_apps.drop([11, 13], axis=1)

# Split into train and test sets
cc_apps_train, cc_apps_test = train_test_split(cc_apps, test_size=0.33, random_state=42)

In [ ]:

import numpy as np


cc_apps_train = cc_apps_train.replace("?", np.NaN)
cc_apps_test = cc_apps_test.replace("?", np.NaN)

In [ ]:
# Impute numeric columns with mean
numeric_columns = cc_apps_train.select_dtypes(include=['float64', 'int64']).columns
cc_apps_train[numeric_columns] = cc_apps_train[numeric_columns].fillna(cc_apps_train[numeric_columns].mean())
cc_apps_test[numeric_columns] = cc_apps_test[numeric_columns].fillna(cc_apps_test[numeric_columns].mean())


cc_apps.isnull().sum()

0     12
1     12
2      0
3      6
4      6
5      9
6      9
7      0
8      0
9      0
10     0
12     0
14     0
15     0
dtype: int64

In [48]:
# Iterate over each column of cc_apps_train
for col in cc_apps_train.columns:
    # Check if the column is of object type
    if cc_apps_train[col].dtypes == 'object':
        # Impute with the most frequent value
        cc_apps_train = cc_apps_train.fillna(cc_apps_train[col].value_counts().index[0])
        cc_apps_test = cc_apps_test.fillna(cc_apps_train[col].value_counts().index[0])

# Count the number of NaNs in the dataset and print the counts to verify
print(cc_apps_train.isnull().sum())
print(cc_apps_test.isnull().sum())

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
12    0
14    0
15    0
dtype: int64
0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
12    0
14    0
15    0
dtype: int64


In [49]:
# Convert the categorical features in the train and test sets independently
cc_apps_train = pd.get_dummies(cc_apps_train)
cc_apps_test = pd.get_dummies(cc_apps_test)

# Reindex the columns of the test set aligning with the train set
cc_apps_test = cc_apps_test.reindex(columns=cc_apps_train.columns, fill_value=0)

In [50]:
# Import MinMaxScaler
from sklearn.preprocessing import MinMaxScaler

# Segregate features and labels into separate variables
X_train, y_train = cc_apps_train.iloc[:, :-1].values, cc_apps_train.iloc[:, [-1]].values
X_test, y_test = cc_apps_test.iloc[:, :-1].values, cc_apps_test.iloc[:, [-1]].values

# Instantiate MinMaxScaler and use it to rescale X_train and X_test
scaler = MinMaxScaler(feature_range=(0, 1))
rescaledX_train = scaler.fit_transform(X_train)
rescaledX_test = scaler.transform(X_test)

In [ ]:
# Import LogisticRegression
from sklearn.linear_model import LogisticRegression


logreg = LogisticRegression()


logreg.fit(rescaledX_train,y_train)

c:\Users\Akshata\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


LogisticRegression()

In [ ]:
# Import confusion_matrix
from sklearn.metrics import confusion_matrix

# Use logreg to predict instances from the test set and store it
y_pred = logreg.predict(rescaledX_test)

# Get the accuracy score of logreg model and print it
print("Accuracy of logistic regression classifier: ", logreg.score(rescaledX_test,y_test))


confusion_matrix(y_test,y_pred)

Accuracy of logistic regression classifier:  1.0


c:\Users\Akshata\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:407: UserWarning: A single label was found in 'y_true' and 'y_pred'. For the confusion matrix to have the correct shape, use the 'labels' parameter to pass all known labels.
  warnings.warn(


array([[229]], dtype=int64)

In [ ]:
# Import GridSearchCV
from sklearn.model_selection import GridSearchCV

# Define the grid of values for tol and max_iter
tol = [0.01, 0.001 ,0.0001]
max_iter = [100, 150, 200]


param_grid = dict(tol=tol, max_iter=max_iter)

In [ ]:

grid_model = GridSearchCV(estimator=logreg, param_grid=param_grid, cv=5)

# Fit grid_model to the data
grid_model_result = grid_model.fit(rescaledX_train, y_train)


best_score, best_params = grid_model_result.best_score_, grid_model_result.best_params_
print("Best: %f using %s" % (best_score, best_params))

# Extract the best modelt
best_model = grid_model_result.best_estimator_
print("Accuracy of logistic regression classifier: ", best_model.score(rescaledX_test,y_test))

c:\Users\Akshata\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\model_selection\_split.py:805: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(
c:\Users\Akshata\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Akshata\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Akshata\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array w

Best: nan using {'max_iter': 100, 'tol': 0.01}
Accuracy of logistic regression classifier:  1.0


c:\Users\Akshata\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Akshata\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Akshata\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
c:\Users\Akshata\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\validation.py:140